In [1]:
# Setup
!pip install -q numpy pandas transformers datasets[audio] accelerate torchaudio jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 re

In [2]:
# Load Dataset
import os
from datasets import Dataset, DatasetDict, Audio

def load_vivos_dataset(base_path):
    all_samples = []
    for split in ["train", "test"]:
        wav_dir = os.path.join(base_path, split, "waves")
        prompt_file = os.path.join(base_path, split, "prompts.txt")
        with open(prompt_file, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                file_id, text = line.split(" ", 1)
                speaker = file_id.split("_")[0]
                wav_path = os.path.join(wav_dir, speaker, file_id + ".wav")
                if os.path.exists(wav_path):
                    all_samples.append({"audio": wav_path, "transcription": text})
    return DatasetDict({"train": Dataset.from_list(all_samples)})

base_path = "/kaggle/input/vivos-vietnamese/vivos"
dataset = load_vivos_dataset(base_path)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [3]:
# Preprocess
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("nguyenvulebinh/wav2vec2-base-vietnamese-250h")

def prepare_batch(batch):
    audio = batch["audio"]
    inputs = processor(audio["array"], sampling_rate=16000, return_tensors="pt", padding=True)

    # Tokenize label riêng
    labels = processor.tokenizer(batch["transcription"], return_tensors="pt", padding=True)

    batch["input_values"] = inputs.input_values[0]
    batch["attention_mask"] = inputs.get("attention_mask", None)
    batch["labels"] = labels.input_ids[0]
    return batch

dataset = dataset.map(prepare_batch, remove_columns=dataset["train"].column_names)

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Map:   0%|          | 0/12420 [00:00<?, ? examples/s]

2025-05-24 18:39:47.262300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748111987.453768      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748111987.507569      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch

@dataclass
class DataCollatorCTCWithPadding:
    processor: Any
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        input_features = [{"input_values": f["input_values"]} for f in features]
        label_features = [{"input_ids": f["labels"]} for f in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt"
        )

        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt"
            )

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels
        return batch

In [5]:
# Train with Accelerate
from transformers import Wav2Vec2ForCTC, TrainingArguments, Trainer

model = Wav2Vec2ForCTC.from_pretrained("nguyenvulebinh/wav2vec2-base-vietnamese-250h")

training_args = TrainingArguments(
    output_dir="./wav2vec2-vivos-ft",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    fp16=True,
    num_train_epochs=10,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    report_to="none",
)

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    tokenizer=processor.feature_extractor,
    data_collator=data_collator
)

config.json:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

/tmp/ipykernel_19/2286904082.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [6]:
config = """
compute_environment: LOCAL_MACHINE
deepspeed_config: {}
distributed_type: MULTI_GPU
downcast_bf16: 'no'
fp16: true
machine_rank: 0
main_process_ip: null
main_process_port: null
main_training_function: main
num_machines: 1
num_processes: 2
use_cpu: false
"""

import os
os.makedirs("/root/.cache/huggingface/accelerate", exist_ok=True)
with open("/root/.cache/huggingface/accelerate/default_config.yaml", "w") as f:
    f.write(config.strip())

In [7]:
# Train
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
50,1.552300
100,0.764700
150,0.524900
200,0.451700
250,0.395500
300,0.391200
350,0.409000
400,0.350700
450,0.320100
500,0.287200


/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/tor

TrainOutput(global_step=7770, training_loss=0.14088691874974177, metrics={'train_runtime': 31595.4352, 'train_samples_per_second': 3.931, 'train_steps_per_second': 0.246, 'total_flos': 1.1122109724923132e+19, 'train_loss': 0.14088691874974177, 'epoch': 10.0})

In [8]:
# Save
model.save_pretrained("/kaggle/working/wav2vec2-vivos-ft")
processor.save_pretrained("/kaggle/working/wav2vec2-vivos-ft")

[]

In [9]:
import shutil
shutil.make_archive("/kaggle/working/wav2vec2-vivos-ft", 'zip', "/kaggle/working/wav2vec2-vivos-ft")

'/kaggle/working/wav2vec2-vivos-ft.zip'

In [10]:
%cd /kaggle/working

/kaggle/working


In [11]:
from IPython.display import FileLink
FileLink(r'wav2vec2-vivos-ft.zip')

/kaggle/working/wav2vec2-vivos-ft.zip